<a href="https://colab.research.google.com/github/hvhq/classificationp1/blob/master/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import path

if not path.exists('gdrive'):
  from google.colab import drive
  drive.mount('gdrive')

In [2]:
#from google.colab import drive
#drive.flush_and_unmount()

In [2]:
import numpy as np
import pandas as pd
import json
import zipfile
import sklearn
import math

In [3]:
!pip install ipython-autotime
%load_ext autotime
!pip install kaggle
!mkdir -p /root/.kaggle

time: 2.54 s


In [4]:
#upload kaggle token file: kaggle.json
#files.upload()
kaggletoken={"username":"huynhvanhien","key":"82d7e5f0d5f149aedf8f040ba7c74c2a"}
with open('/root/.kaggle/kaggle.json', 'w')as kaggletokenfile:
  json.dump(kaggletoken, kaggletokenfile)
#move to /.kaggle

#!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

time: 121 ms


In [5]:
!kaggle datasets download -d mohansacharya/graduate-admissions

graduate-admissions.zip: Skipping, found more recently modified local copy (use --force to force download)
time: 718 ms


In [6]:
zipreference=zipfile.ZipFile('graduate-admissions.zip', 'r')
zipreference.extractall('Data')
zipreference.close()

time: 7.4 ms


In [33]:
data=pd.read_csv("Data/Admission_Predict_Ver1.1.csv")
#remove unused columns
if 'Serial No.' in data.columns:
  data=data.drop(['Serial No.'], axis=1)
#insert bias column
biasc=pd.Series([int(1) for i in range(len(data))])
#data.insert(0, "biascol", biasc, True)
data['biascol']=biasc

#filter one species
#datab=pd.DataFrame(columns=['Species','Weight','Length1','Length2', 'Length3', 'Height', 'Width'])
#for i in range(0, lendata):
#  if data.Species[i]=='Bream':
#    datab=datab.append(data.iloc[i], ignore_index=True)
#data=datab
#lendata=len(data)

#encoder string to numeric
#encoder=sklearn.preprocessing.LabelEncoder()
#encoder.fit(data.Fuel_Type)
#data.Fuel_Type=encoder.transform(data.Fuel_Type)
#encoder.fit(data.Transmission)
#data.Transmission=encoder.transform(data.Transmission)

lendata=len(data)
#change domain to [0,1]
for i in range(0, len(data.columns)):
  #print(data.iloc[:,i])
  if max(data.iloc[:,i])-min(data.iloc[:,i])!=0:
    data.iloc[:,i]=(data.iloc[:,i]-([min(data.iloc[:,i])]*lendata))/([max(data.iloc[:,i])-min(data.iloc[:,i])]*lendata)
  else:
    data.iloc[:,i]=1
print(data)

     GRE Score  TOEFL Score  ...  Chance of Admit   biascol
0         0.94     0.928571  ...          0.920635        1
1         0.68     0.535714  ...          0.666667        1
2         0.52     0.428571  ...          0.603175        1
3         0.64     0.642857  ...          0.730159        1
4         0.48     0.392857  ...          0.492063        1
..         ...          ...  ...               ...      ...
495       0.84     0.571429  ...          0.841270        1
496       0.94     0.892857  ...          0.984127        1
497       0.80     1.000000  ...          0.936508        1
498       0.44     0.392857  ...          0.619048        1
499       0.74     0.750000  ...          0.793651        1

[500 rows x 9 columns]
time: 48.8 ms


In [34]:
#split to x, y train and test
data.columns=data.columns.str.replace(' ', '_')
x=data.drop('Chance_of_Admit_', axis=1)
y=pd.DataFrame(data['Chance_of_Admit_'])
trainOTestRate=3
xtrain=x
xtest=pd.DataFrame(columns=x.columns)
ytrain=y
ytest=pd.DataFrame(columns=['Chance_of_Admit_'])
for i in range(0, len(data)):
  if i%(trainOTestRate+1)==0:
    xtest=xtest.append(x.iloc[i], ignore_index=True)
    xtrain=xtrain.drop([i])
    ytest=ytest.append(y.iloc[i], ignore_index=True)
    ytrain=ytrain.drop([i])
xtrain=xtrain.reset_index(drop=True)
ytrain=ytrain.reset_index(drop=True)

#xtrain=xtrain.to_numpy()
#xtest=xtest.to_numpy()
#ytrain=ytrain.to_numpy()
#ytest=ytest.to_numpy()
print("train")
print(xtrain)
print(ytrain)
print("test")
print(xtest)
print(ytest)

#convert y to vector
ytrain=ytrain.iloc[:,0]
ytest=ytest.iloc[:,0]
lenTrain=len(xtrain)
numberInputAttr=len(x.columns)
vector0=np.array([0]*numberInputAttr)
xtrain=np.array(xtrain)
xtest=np.array(xtest)

train
     GRE_Score  TOEFL_Score  University_Rating  ...      CGPA  Research  biascol
0         0.68     0.535714               0.75  ...  0.663462       1.0        1
1         0.52     0.428571               0.50  ...  0.384615       1.0        1
2         0.64     0.642857               0.50  ...  0.599359       1.0        1
3         0.80     0.821429               1.00  ...  0.814103       1.0        1
4         0.62     0.607143               0.50  ...  0.448718       1.0        1
..         ...          ...                ...  ...       ...       ...      ...
370       0.22     0.250000               0.50  ...  0.528846       1.0        1
371       0.84     0.571429               1.00  ...  0.711538       1.0        1
372       0.80     1.000000               1.00  ...  0.884615       1.0        1
373       0.44     0.392857               0.75  ...  0.522436       0.0        1
374       0.74     0.750000               0.75  ...  0.717949       0.0        1

[375 rows x 8 columns

In [ ]:
#split train and test using ski-learn
#from sklearn.cross_validation import train_test_split
#x=data.drop('Chance_of_Admit_', axis=1)
#y=pd.DataFrame(data['Chance_of_Admit_'])
#xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=0)

In [35]:
#locally weighted regression
lenTrain=len(xtrain)
alpha=0.01
numEpochs=10000

msa=0
for i in range(len(xtest)):
  theta=vector0*1
  numLoops=0
  while numLoops<numEpochs:
    error=np.dot(xtrain[numLoops%lenTrain], np.transpose(theta))-ytrain[numLoops%lenTrain]
    total_errors=np.dot(np.transpose(error), xtrain[numLoops%lenTrain])

    #local_weight=math.exp(-(xtrain[numLoops%lenTrain]-xtest)**2/2)
    xdistance=xtrain[numLoops%lenTrain]-xtest[i]
    xdistance=-xdistance * xdistance / 2
    local_weight=[math.exp(i) for i in xdistance]

    theta=theta-local_weight*(alpha*total_errors)
    numLoops+=1

  #predict:
  ypredict=np.dot(theta, xtest[i])
  print('predict', i, ':', ypredict, 'error:',  ypredict-ytest[i])
  print('ytest i:', ytest[i])
  msa+=abs(ypredict-ytest[i])/len(xtest)

#evaluate
print('1-msa:',1-msa)

predict 0 : 0.9931288364452722 error: 0.07249391581035125
ytest i: 0.9206349206349209
predict 1 : 0.4774031150301521 error: -0.014660377033340055
ytest i: 0.49206349206349215
predict 2 : 0.3362769018399407 error: 0.08230864787168674
ytest i: 0.25396825396825395
predict 3 : 0.843518498767977 error: 0.14510580035527854
ytest i: 0.6984126984126985
predict 4 : 0.6124995593311415 error: 0.10456305139463351
ytest i: 0.507936507936508
predict 5 : 0.49973322860555186 error: 0.023542752415075585
ytest i: 0.4761904761904763
predict 6 : 0.9977041994095817 error: -0.00229580059041834
ytest i: 1.0
predict 7 : 0.16760997487479315 error: -0.022866215601397344
ytest i: 0.1904761904761905
predict 8 : 0.9548648329449367 error: 0.05010292818303175
ytest i: 0.904761904761905
predict 9 : 0.5205562541925954 error: 0.04436577800211916
ytest i: 0.4761904761904763
predict 10 : 0.5552277879016431 error: 0.3647515974254526
ytest i: 0.1904761904761905
predict 11 : 0.8867482893938118 error: -0.018013615368093205
y

In [45]:
#logistic regression
#batch gradient descent
#print(x)
#print(y)
theta=vector0*1

#train
alpha=0.01
numEpochs=10000
numLoops=0
while(numLoops<numEpochs):
  jtheta=np.dot(xtrain, np.transpose(theta))
  htheta=[1/(1+math.exp(-i)) for i in jtheta]
  errors=ytrain-htheta#np.dot(xtrain, np.transpose(theta))
  sumerrors=np.dot(np.transpose(errors), xtrain)
  theta=theta+alpha*sumerrors
  numLoops+=1

#result
print('theta: ',theta)

#test
test_results=np.dot(xtest, np.transpose(theta))
test_errors=np.dot(xtest, np.transpose(theta))-ytest
com_res=pd.DataFrame(test_results, columns=['cal_result'])
com_res.insert(1, 'check', ytest, True)
com_res.insert(2, 'error', test_errors, True)
print(com_res)
print('Evaluate:')
mse=1.0/len(ytest)*np.sum([i*i for i in com_res.error])
mae=1.0/len(ytest)*np.sum([i if i>=0 else -i for i in com_res.error])
print('1-sqrt(mse):', 1-math.sqrt(mse))
print('1-mae:', 1-mae)

theta:  [ 0.56211431  0.73030843  0.24450684 -0.0437573   0.52831225  2.98234779
  0.16252583 -2.38465746]
     cal_result     check     error
0      2.316025  0.920635  1.395390
1     -0.165800  0.492063 -0.657864
2     -0.786769  0.253968 -1.040737
3      1.538085  0.698413  0.839672
4      0.479886  0.507937 -0.028051
..          ...       ...       ...
120    0.990839  0.730159  0.260681
121   -0.152464  0.619048 -0.771512
122    0.895582  0.666667  0.228916
123   -0.417473  0.301587 -0.719060
124    2.621932  0.984127  1.637805

[125 rows x 3 columns]
Evaluate:
1-sqrt(mse): 0.14536535143438667
1-mae: 0.27976119960396106
time: 3.83 s
